# Process Dataset without meta_window

In [1]:
import cv2
import os
import re
import dlib
from scipy.spatial import distance
import numpy as np
import pandas as pd

In [2]:
# Función para calcular EAR
def calculate_ear(eye_points):
    A = distance.euclidean(eye_points[1], eye_points[5])
    B = distance.euclidean(eye_points[2], eye_points[4])
    C = distance.euclidean(eye_points[0], eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [3]:
# Función para calcular MAR
def calculate_mar(mouth_points):
    A = distance.euclidean(mouth_points[2], mouth_points[8])
    B = distance.euclidean(mouth_points[3], mouth_points[7])
    C = distance.euclidean(mouth_points[4], mouth_points[6])
    D = distance.euclidean(mouth_points[0], mouth_points[5])
    mar = (A + B + C) / (2.0 * D)
    return mar

In [4]:
# Función para estimar ángulo de cabeza
def calculate_head_angle(landmarks):
    model_points = np.array([
        (0.0, 0.0, 0.0),  # Nariz
        (0.0, -330.0, -65.0),  # Barbilla
        (-225.0, 170.0, -135.0),  # Ojo izquierdo
        (225.0, 170.0, -135.0),  # Ojo derecho
        (-150.0, -150.0, -125.0),  # Boca izquierda
        (150.0, -150.0, -125.0)  # Boca derecha
    ])
    image_points = np.array([
        (landmarks.part(30).x, landmarks.part(30).y),  # Nariz
        (landmarks.part(8).x, landmarks.part(8).y),  # Barbilla
        (landmarks.part(36).x, landmarks.part(36).y),  # Ojo izquierdo
        (landmarks.part(45).x, landmarks.part(45).y),  # Ojo derecho
        (landmarks.part(48).x, landmarks.part(48).y),  # Boca izquierda
        (landmarks.part(54).x, landmarks.part(54).y)  # Boca derecha
    ], dtype="double")
    
    focal_length = 640  # Aproximado para webcam
    center = (640 / 2, 480 / 2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype="double")
    dist_coeffs = np.zeros((4, 1))
    _, rotation_vector, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    angle = np.linalg.norm(rotation_vector) * 180 / np.pi
    return angle

In [5]:
# Función para contar parpadeos
def count_blinks(ears, ear_threshold=0.2):
    blinks = 0
    prev_ear = ears[0]
    for ear in ears[1:]:
        if prev_ear > ear_threshold and ear <= ear_threshold:
            blinks += 1
        prev_ear = ear
    return blinks

In [6]:
# Función para extraer características de una imagen
def extract_features_from_image(image_path, detector, predictor):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    faces = detector(img)
    for face in faces:
        landmarks = predictor(img, face)
        left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
        right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]
        mouth = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)]
        ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0
        mar = calculate_mar(mouth)
        head_angle = calculate_head_angle(landmarks)
        return {'ear': ear, 'mar': mar, 'head_angle': head_angle}
    return None

In [7]:
# Función para procesar el dataset
def process_dataset(drowsy_dir, notdrowsy_dir, detector, predictor, window_size=20):
    videos = {}
    # Procesar carpeta drowsy
    for img_name in os.listdir(drowsy_dir):
        match = re.match(r"(\d+_\w+_\w+)_(\d+)_drowsy\.jpg", img_name)
        if match:
            video_id_base, frame_num = match.groups()
            video_id = f"{video_id_base}_drowsy"
            if video_id not in videos:
                videos[video_id] = {'frames': [], 'label': 1}
            videos[video_id]['frames'].append((int(frame_num), os.path.join(drowsy_dir, img_name)))
    
    # Procesar carpeta notdrowsy
    for img_name in os.listdir(notdrowsy_dir):
        match = re.match(r"(\d+_\w+_\w+)_(\d+)_notdrowsy\.jpg", img_name)
        if match:
            video_id_base, frame_num = match.groups()
            video_id = f"{video_id_base}_notdrowsy"
            if video_id not in videos:
                videos[video_id] = {'frames': [], 'label': 0}
            videos[video_id]['frames'].append((int(frame_num), os.path.join(notdrowsy_dir, img_name)))
    
    # Ordenar frames y extraer características
    data = []
    for video_id, info in videos.items():
        info['frames'].sort()  # Ordenar por número de fotograma
        print(f"Video {video_id}: Inicio={info['frames'][0][0]}, Fin={info['frames'][-1][0]}, Etiqueta={info['label']}, Imágenes={len(info['frames'])}")
        
        ears, mars, head_angles = [], [], []
        for _, frame_path in info['frames']:
            features = extract_features_from_image(frame_path, detector, predictor)
            if features:
                ears.append(features['ear'])
                mars.append(features['mar'])
                head_angles.append(features['head_angle'])
        
        if len(ears) >= window_size:
            for i in range(0, len(ears) - window_size + 1):
                window_ears = ears[i:i + window_size]
                window_mars = mars[i:i + window_size]
                window_angles = head_angles[i:i + window_size]
                blink_freq = count_blinks(window_ears)
                data.append({
                    'video_id': video_id,
                    'ear_mean': np.mean(window_ears),
                    'ear_std': np.std(window_ears),
                    'ear_min': np.min(window_ears),
                    'mar_mean': np.mean(window_mars),
                    'mar_std': np.std(window_mars),
                    'mar_max': np.max(window_mars),
                    'head_angle_mean': np.mean(window_angles),
                    'blink_freq': blink_freq,
                    'label': info['label']
                })
    
    return pd.DataFrame(data)

In [8]:
# Directorios del dataset
drowsy_dir = "dataset/drowsy"
notdrowsy_dir = "dataset/notdrowsy"

# Cargar detector y predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("face_landmarks/shape_predictor_68_face_landmarks.dat")  # Descarga este archivo

# Procesar dataset
df = process_dataset(drowsy_dir, notdrowsy_dir, detector, predictor)
df.to_csv("features_without_metawindow.csv", index=False)
print("Características guardadas en features_without_metawindow.csv")

Video 001_glasses_sleepyCombination_drowsy: Inicio=599, Fin=2747, Etiqueta=1, Imágenes=2149
Video 001_glasses_slowBlinkWithNodding_drowsy: Inicio=464, Fin=1920, Etiqueta=1, Imágenes=1217
Video 001_glasses_yawning_drowsy: Inicio=339, Fin=1849, Etiqueta=1, Imágenes=1007
Video 001_noglasses_sleepyCombination_drowsy: Inicio=330, Fin=2710, Etiqueta=1, Imágenes=2381
Video 001_noglasses_slowBlinkWithNodding_drowsy: Inicio=398, Fin=1851, Etiqueta=1, Imágenes=1154
Video 001_noglasses_yawning_drowsy: Inicio=196, Fin=1871, Etiqueta=1, Imágenes=1676
Video 002_glasses_sleepyCombination_drowsy: Inicio=314, Fin=2878, Etiqueta=1, Imágenes=2565
Video 002_glasses_slowBlinkWithNodding_drowsy: Inicio=395, Fin=2035, Etiqueta=1, Imágenes=1544
Video 002_glasses_yawning_drowsy: Inicio=432, Fin=2066, Etiqueta=1, Imágenes=1635
Video 002_noglasses_sleepyCombination_drowsy: Inicio=239, Fin=2656, Etiqueta=1, Imágenes=2418
Video 002_noglasses_slowBlinkWithNodding_drowsy: Inicio=209, Fin=1809, Etiqueta=1, Imágenes=1